In [51]:
import uuid

import pandas as pd

In [52]:
draft_df = pd.read_excel('data/anketa_2.xlsx')

# Предварительная подготовка таблиц #

In [53]:
display(draft_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype         
---  ------                                                            --------------  -----         
 0   Отметка времени                                                   320 non-null    datetime64[ns]
 1   Профиль телеграмм (в формате @username)                           321 non-null    object        
 2   Часовой пояс                                                      322 non-null    object        
 3   Стек технологий                                                   321 non-null    object        
 4   По какой специальности SF хотите заявиться?                       322 non-null    object        
 5   Роль                                                              322 non-null    object        
 6   В какой роли видите себя в проекте?                               322 non-

None

#### Переименование признаков для размещения таблиц в БД ####

In [54]:
old_cols = [
    'Отметка времени',
    'Профиль телеграмм (в формате @username)',
    'Часовой пояс',
    'Стек технологий',
    'По какой специальности SF хотите заявиться?',
    'Роль',
    'В какой роли видите себя в проекте?',
    'Сколько часов в неделю готовы уделять проекту?',
    'Какие другие курсы закончили или находитесь в процессе обучения?',
    'Как долго учитесь на курсах?',
    'Notes',
    'ЯП',
    'Вступление в чат практики.'
]
new_cols = [
    'date',
    'tg_id',
    'timezone',
    'stack',
    'occupation',
    'role',
    'project_role',
    'weekload',
    'course',
    'course_time',
    'notes',
    'pl',
    'chat'
]

def map_names(old_list: list, new_list: list) -> dict:
    dict_ = dict()
    if len(old_list) == len(new_list):
        for old_name, new_name in zip(old_list, new_list):
            dict_[old_name] = new_name
    return dict_

draft_df.rename(columns=map_names(old_cols, new_cols), inplace=True)
draft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          320 non-null    datetime64[ns]
 1   tg_id         321 non-null    object        
 2   timezone      322 non-null    object        
 3   stack         321 non-null    object        
 4   occupation    322 non-null    object        
 5   role          322 non-null    object        
 6   project_role  322 non-null    object        
 7   weekload      322 non-null    object        
 8   course        322 non-null    object        
 9   course_time   322 non-null    object        
 10  notes         99 non-null     object        
 11  pl            205 non-null    object        
 12  chat          322 non-null    object        
dtypes: datetime64[ns](1), object(12)
memory usage: 32.8+ KB


#### Контроль пустых мест ####

In [55]:
display(draft_df.isna().sum(axis='index'))

date              2
tg_id             1
timezone          0
stack             1
occupation        0
role              0
project_role      0
weekload          0
course            0
course_time       0
notes           223
pl              117
chat              0
dtype: int64

In [56]:
draft_df.tg_id.nunique()

321

#### Выделение идентификаторов пользователей в отдельную таблицу **`private_table.csv`** ####

In [57]:
draft_df['uuid'] = ''
draft_df.uuid = draft_df.uuid.transform(lambda x: str(uuid.uuid4()))
private_table_df = draft_df[['uuid', 'tg_id']]
private_table_df.to_csv('data/private_table.csv')

#### Удаление из основной таблицы telegram-логинов (остаются только **`uuid`**) ####

In [58]:
table_df = draft_df.drop('tg_id', axis='columns')
table_df.to_csv('data/table.csv')

In [59]:
table_df.describe(include='object')

,timezone,stack,occupation,role,project_role,weekload,course,course_time,notes,pl,chat,uuid
count,322,321,322,322,322,322,322,322,99,205,322,322
unique,7,260,84,13,251,3,309,111,18,26,3,322
top,GMT+3 (Москва),Python,Тестирование,Тестировщик,Тестировщик,20+ часов,Skillbox,Заканчиваю,manual,Python,Да,bd37bc23-ebc6-4407-8000-aa8101c80c12
freq,246,21,86,93,17,135,4,65,47,39,243,1
